In [1]:
#importing the neccessary libraries
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import random
from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate
from sklearn.metrics import f1_score, accuracy_score
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from collections import Counter

Using TensorFlow backend.


In [2]:
#reading preprocessed training and test datasets using pandas
df_train = pd.read_csv("mitbih_train.csv", header=None)
df_train = df_train.sample(frac=1)
df_test = pd.read_csv("mitbih_test.csv", header=None)

#get train & test data in desired datatype
Y = np.array(df_train[187].values).astype(np.int8)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]
Y_test = np.array(df_test[187].values).astype(np.int8)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]

In [3]:
#model function
def get_model():
    #number of categories in our problem
    nclass = 5
    #shape of input
    inp = Input(shape=(187, 1))
    #activations allows models to take into account non-linear relationships. relu = rectified linear activation
    #kernel is the matrix of weights that is convolved on the input. Often odd numbers (5 and 3 in this case)
    #pooling is to downsample the input
    #1 -- 16 filters
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(inp)
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1) #to prevent overfitting
    #2 -- 16x2 filters
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1) #to prevent overfitting
    #3 -- 16x2 filters
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1) #to prevent overfitting
    #4 -- 16x16 filters
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = GlobalMaxPool1D()(img_1)
    img_1 = Dropout(rate=0.2)(img_1) #to prevent overfitting
    
    #Dense is the layer type. 
    #Dense is a standard layer type that works for most cases. 
    #In a dense layer, all nodes in the previous layer connect to the nodes in the current layer.
    dense_1 = Dense(64, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = Dense(64, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = Dense(nclass, activation=activations.softmax, name="dense_3_mitbih")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.sparse_categorical_crossentropy, metrics=['acc'])
    model.summary()
    return model

In [4]:
#get the model and save it in an h5 binary file
model = get_model()
file_path = "baseline_cnn_mitbih.h5"

#checkpointing the model's weight based on the accuracy of the model
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

#set early stopping based on accuracy improving or not. It stops after 5 epochs of no accuracy improvement
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)

#reduces learning rate when a metric has stopped improving
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)

#defining the callbacks list to include the above parameters
callbacks_list = [checkpoint, early, redonplat]

#train the model
model.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model.load_weights(file_path)

#test the model
pred_test = model.predict(X_test)
pred_test = np.argmax(pred_test, axis=-1)

#get f1 score of the model & print it. The f1 score considers the precision & recall.
f1 = f1_score(Y_test, pred_test, average="macro")
print("Test f1 score : %s "% f1)

#get accuracy score of the model & print it
acc = accuracy_score(Y_test, pred_test)
print("Test accuracy score : %s "% acc)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 187, 1)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 183, 16)           96        
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 179, 16)           1296      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 89, 16)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 89, 16)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 87, 32)            1568      
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 85, 32)            3104

In [ ]:
# #randomly selects 10 records and displays their graphs. an observation is that because there are so many normal records, it often selects them which is simple probability
# df = pd.read_csv("mitbih_test.csv", header=None)
# print(df.shape)
# print(Counter(df[187].values))

# Y = np.array(df[187].values).astype(np.int8)
# X = np.array(df[list(range(187))].values)

# indexes = random.sample(list(range(df.shape[0])), 10)

# for i in indexes:

#     data = [go.Scatter(
#               x=list(range(187)),
#               y=X[i, :])]

#     plot({"data": data,
#           "layout": {"title": "Heartbeat Class : %s "%Y[i]}}, filename='%s.html'%i)

In [8]:
#testing the model on abnormal cases only (categories 1,2,3,4)
df_test_ab = pd.read_csv("/Users/lujainmohd99/Desktop/ecg-classification/attempt2/ECG_Heartbeat_Classification/code/abs.csv", header=None)

Y_test_ab = np.array(df_test_ab[187].values).astype(np.int8)
X_test_ab = np.array(df_test_ab[list(range(187))].values)[..., np.newaxis]

#test the model
pred_test_ab = model.predict(X_test_ab)
pred_test_ab = np.argmax(pred_test_ab, axis=-1)

#get f1 score of the model & print it. The f1 score considers the precision & recall.
f1_ab = f1_score(Y_test_ab, pred_test_ab, average="macro")
print("Test f1 score : %s "% f1_ab)

#get accuracy score of the model & print it
acc_ab = accuracy_score(Y_test_ab, pred_test_ab)
print("Test accuracy score : %s "% acc_ab)

Test f1 score : 0.7243043646463921 
Test accuracy score : 0.9250132485426603 
